In [1]:
# %pip install datasets
# %pip install Pillow
# %pip install numpy
# %pip install matplotlib
# %pip install import_ipynb
# %pip install tensorflow
# %pip install tensorflow.keras
# %pip install easyocr
# %pip install pandas

# # library from etl
# %pip install datasets
# %pip install numpy
# %pip install Pillow
# %pip install opencv-python
# %pip install json

In [2]:
import easyocr
import matplotlib.pyplot as plt
from datasets import load_dataset
import numpy as np
from PIL import Image
import cv2
import tensorflow as tf
# from tensorflow.keras import layers
import json
import pandas as pd

In [3]:
from kaggle_secrets import UserSecretsClient
import os

# Ambil GitHub token dari Kaggle Secrets
user_secrets = UserSecretsClient()
myGithub = user_secrets.get_secret("githubKey")

# Clone repo GitHub
!git clone https://{myGithub}@github.com/fitristachan/invoice-image-processing.git invoice_image_processing
!git -C invoice_image_processing pull

fatal: destination path 'invoice_image_processing' already exists and is not an empty directory.
Already up to date.


In [4]:
# from google.colab import userdata

# # Retrieve the GitHub token from userdata
# myGithub = userdata.get('githubKey')

# !git clone https://{myGithub}@github.com/fitristachan/invoice-image-processing.git /content/invoice_image_processing
# !git -C invoice_image_processing pull



In [5]:
# import sys
# import os

# sys.path.append('/content/invoice-image-processing')
# print(os.listdir('/content/invoice-image-processing'))

# !jupyter nbconvert --to script '/content/invoice-image-processing/data_etl_invoice.ipynb' --output data_etl

Image Classification

In [6]:
from invoice_image_processing.data_etl import DatasetReceipt

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [7]:
# Inisialisasi dataset
dataset_local = DatasetReceipt(dataset_name="naver-clova-ix/cord-v2", split="train")
dataset_inter = DatasetReceipt(dataset_name="katanaml-org/invoices-donut-data-v1", split="train")

In [8]:
from keras.applications import ResNet50V2

# keras.applications.ResNet50V2(
#     include_top=True,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation="softmax",
#     name="resnet50v2",
# )

pre_trained_model = ResNet50V2(input_shape = (600, 600, 3),
                                include_top = False,
                                pooling='max'
                                )

pre_trained_model.summary()

Model: "resnet50v2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 600, 600, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 606, 606, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 300, 300, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 302, 302, 64)   │              0 │ conv1_conv[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 150, 150, 64)   │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_preact_bn    │ (None, 150, 150, 64)   │            256 │ pool1_pool[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_preact_relu  │ (None, 150, 150, 64)   │              0 │ conv2_block1_preact_b… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 150, 150, 64)   │          4,096 │ conv2_block1_preact_r… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 150, 150, 64)   │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 150, 150, 64)   │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_pad        │ (None, 152, 152, 64)   │              0 │ conv2_block1_1_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 150, 150, 64)   │         36,864 │ conv2_block1_2_pad[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 150, 150, 64)   │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 150, 150, 64)   │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 150, 150, 256)  │         16,640 │ conv2_block1_preact_r… │
│ (Conv2D)             

 Total params: 23,564,800 (89.89 MB)

 Trainable params: 23,519,360 (89.72 MB)

 Non-trainable params: 45,440 (177.50 KB)

In [9]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Load Dataset (Gunakan DatasetReceipt class yang sudah dibuat sebelumnya)
train_dataset = DatasetReceipt(dataset_name="naver-clova-ix/cord-v2", split="train")
# val_dataset = DatasetReceipt(dataset_name="katanaml-org/invoices-donut-data-v1", split="train")
val_dataset = DatasetReceipt(dataset_name="naver-clova-ix/cord-v2", split="validation")

def find_price_column(df):
    # Prioritas 1: Kolom dengan nama exact 'price'
    if 'price' in df.columns:
        return 'price'
    
    # Prioritas 2: Cari kolom mengandung 'price' atau 'unitprice'
    price_columns = [
        col for col in df.columns 
        if any(kwd in col.lower() for kwd in ['price', 'unitprice'])
    ]
    
    return price_columns[0] if price_columns else None

# Function to generate batches
def dataset_generator(dataset, batch_size=16, max_items=10):
    while True:
        # Initialize batch containers
        batch_images = []
        batch_bboxes = []
        batch_classes = []
        batch_prices = []
        batch_totals = []

        for _ in range(batch_size):
            sample = dataset[np.random.randint(len(dataset))]
            
            # 1. Process Image
            img = sample["image"]
            if img.ndim == 4 and img.shape[0] == 1:
                img = img[0]  # Remove batch dimension if present
            batch_images.append(img)

            # 2. Bounding Box (mock data - replace with actual coordinates from your dataset)
            # Normalized coordinates [x1, y1, x2, y2]
            bbox = [0.1, 0.1, 0.5, 0.5]  # Replace with actual bbox if available
            batch_bboxes.append(bbox)

            # 3. Class Label (one-hot encoded)
            # Assuming 5 classes - replace with actual class if available
            class_label = to_categorical(0, num_classes=5)  # Using class 0 as example
            batch_classes.append(class_label)

            # 4. Price Regression (critical fix)
            padded_prices = np.zeros(max_items)  # Default all zeros
            
            if "menu_df" in sample and not sample["menu_df"].empty:
                menu_df = sample["menu_df"]
                
                # Find price column with flexible matching
                price_col = None
                for col in menu_df.columns:
                    if 'price' in col.lower():
                        price_col = col
                        break
                
                if price_col is not None:
                    try:
                        # Clean and convert prices
                        prices = pd.to_numeric(menu_df[price_col], errors='coerce')
                        prices = prices.fillna(0).values  # Replace NaN with 0
                        prices = np.clip(prices, 0, None)  # Clip negative values
                        
                        # Pad to max_items
                        n_items = min(len(prices), max_items)
                        padded_prices[:n_items] = prices[:n_items]
                    except Exception as e:
                        print(f"Price processing error: {e}")
                        # Keep zeros if error occurs
            
            batch_prices.append(padded_prices)

            # 5. Total Regression
            total = max(0, float(sample.get("total_price", 0)))  # Ensure positive float
            batch_totals.append([total])

        # Convert to numpy arrays and yield
        yield (
            np.array(batch_images, dtype=np.float32),
            {
                "bounding_box": np.array(batch_bboxes, dtype=np.float32),
                "class_label": np.array(batch_classes, dtype=np.float32),
                "price_regression": np.array(batch_prices, dtype=np.float32),
                "total_regression": np.array(batch_totals, dtype=np.float32)
            }
        )

In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras import regularizers

# 1. Define Input Layer
inputs = Input(shape=(600, 600, 3))  # Match your image size

# 2. Load Pretrained ResNet50V2 (without top)
base_model = ResNet50V2(
    input_shape=(600, 600, 3),
    include_top=False,
    pooling='max'  # GlobalMaxPooling2D
)
base_model.trainable = False  # Freeze weights

# 3. Connect Base Model to Inputs
x = base_model(inputs)

# 4. Add Custom Layers
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)

# 5. Define Output Layers
bbox_output = Dense(4, name="bounding_box")(x)  # 4 coordinates
class_output = Dense(5, activation='softmax', name="class_label")(x)
price_output = Dense(10, activation='linear', name='price_regression')(x)
total_output = Dense(1, name="total_regression")(x)  # Predict total

# 6. Create Model
model = Model(
    inputs=inputs,
    outputs=[bbox_output, class_output, price_output, total_output]
)

# 7. Compile
model.compile(
    # optimizer='adam',
    optimizer = Adam(learning_rate=0.0001),
    loss={
        "bounding_box": "mse",
        "class_label": "categorical_crossentropy",
        "price_regression": "mse",
        "total_regression": "mse"
    },
    metrics={
        'class_label': 'accuracy',  # Ini yang akan menghasilkan kolom akurasi
        'bounding_box': 'mae',
        'total_regression': 'mae'
    }
)

In [11]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 600, 600, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ resnet50v2 (Functional)   │ (None, 2048)           │     23,564,800 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 512)            │      1,049,088 │ resnet50v2[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 512)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 512)            │        262,656 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bounding_box (Dense)      │ (None, 4)              │          2,052 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ class_label (Dense)       │ (None, 5)              │          2,565 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ price_regression (Dense)  │ (None, 10)             │          5,130 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ total_regression (Dense)  │ (None, 1)              │            513 │ dense_1[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 24,886,804 (94.94 MB)

 Trainable params: 1,322,004 (5.04 MB)

 Non-trainable params: 23,564,800 (89.89 MB)

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

checkpoint_path = "/content/drive/MyDrive/dataset/model/model_checkpoint_resnet50v2.keras"  # Ganti .h5 → .keras
checkpoint_callback = ModelCheckpoint(checkpoint_path, save_best_only=False)

In [13]:
class Callbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy'] > 0.95 and logs['val_accuracy'] > 95:
            print("\nAccuracy reached 95%! Stopping training.")
            self.model.stop_training = True

stop_callback = Callbacks()

In [14]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('training_log.csv', separator=',', append=True)

In [15]:
# Data Augmentation
train_generator = dataset_generator(train_dataset, batch_size=16)
val_generator = dataset_generator(val_dataset, batch_size=16)

In [18]:
# # Train Model
# model.fit(train_generator,
#           validation_data=val_generator,
#           epochs=25,
#           steps_per_epoch=1000,
#           validation_steps=200,
#           callbacks=[checkpoint_callback, csv_logger])

model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,
    steps_per_epoch=1000,
    validation_steps=200,
    callbacks=[checkpoint_callback, csv_logger]
)

Epoch 1/25
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1202s 1s/step - bounding_box_loss: 37468.6484 - bounding_box_mae: 141.6307 - class_label_accuracy: 1.0000 - class_label_loss: 0.0000e+00 - loss: 109655629824.0000 - price_regression_loss: 41921424.0000 - total_regression_loss: 109613678592.0000 - total_regression_mae: 117315.2500 - val_bounding_box_loss: 1716.5610 - val_bounding_box_mae: 34.2506 - val_class_label_accuracy: 1.0000 - val_class_label_loss: 0.0000e+00 - val_loss: 70855360512.0000 - val_price_regression_loss: 111656.6797 - val_total_regression_loss: 70855229440.0000 - val_total_regression_mae: 119659.5938
Epoch 2/25
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1186s 1s/step - bounding_box_loss: 56570.3750 - bounding_box_mae: 185.9331 - class_label_accuracy: 1.0000 - class_label_loss: 0.0000e+00 - loss: 104857616384.0000 - price_regression_loss: 36041932.0000 - total_regression_loss: 104821514240.0000 - total_regression_mae: 131827.0000 - val_bounding_box_loss: 2286.1694 - val_bounding_box_mae: 41

KeyboardInterrupt: 

In [ ]:
print("Kolom yang tersedia di CSV:")
print(log_data.columns.tolist())

with open("training_log.csv") as f:
    for i, line in enumerate(f):
        if i == 0: 
            print("Header:", len(line.split(',')), "columns")
        if i == 26:  # Baris 27 (indeks 0-based)
            print("Line 27:", len(line.split(',')), "columns")
            print("Content:", line)
            break

In [ ]:
from io import StringIO
import pandas as pd
import matplotlib.pyplot as plt

def clean_log_file(path):
    with open(path) as f:
        # Ambil header (baris pertama)
        header = f.readline().strip()
        # Filter baris yang memiliki 12 koma (13 kolom)
        lines = [line.strip() for line in f if line.count(',') == 12]
        
    # Gabungkan header dengan data yang sudah difilter
    cleaned_data = [header] + lines
    return pd.read_csv(StringIO('\n'.join(cleaned_data)))

# Baca dan bersihkan data
try:
    log_data = clean_log_file("training_log.csv")
except Exception as e:
    print(f"Error: {e}")
    # Fallback: baca dengan engine python
    log_data = pd.read_csv("training_log.csv", engine='python', on_bad_lines='warn')

In [ ]:
# Load training log
log_data = pd.read_csv("training_log.csv")

plt.figure(figsize=(15, 10))

# Plot Total Loss
plt.subplot(2, 2, 1)
plt.plot(log_data['epoch'], log_data['loss'], 'b-', label='Training')
plt.plot(log_data['epoch'], log_data['val_loss'], 'r-', label='Validation')
plt.title('Total Loss')
plt.legend()

# Plot Regression Performance
plt.subplot(2, 2, 2)
plt.plot(log_data['epoch'], log_data['total_regression_mae'], 'g-', label='Train MAE')
plt.plot(log_data['epoch'], log_data['val_total_regression_mae'], 'm-', label='Val MAE')
plt.title('Total Regression MAE')
plt.legend()

# Plot Component Losses
plt.subplot(2, 2, 3)
plt.plot(log_data['epoch'], log_data['bounding_box_loss'], 'c-', label='BBox Loss')
plt.plot(log_data['epoch'], log_data['price_regression_loss'], 'y-', label='Price Loss')
plt.title('Component Losses')
plt.legend()

# Plot Class Loss
plt.subplot(2, 2, 4)
plt.plot(log_data['epoch'], log_data['class_label_loss'], 'k-', label='Class Loss')
plt.title('Classification Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# # Save Model
# model.save("faster_rcnn_resnet50v2.h5")
# print("Model saved successfully!")

**Text Extraction**

> using easyocr

> support bahasa indonesia and english





In [ ]:
reader = easyocr.Reader(['en', 'id'])

def is_receipt(text):
    text_lower = text.lower()

    if 'total' or 'amount' or 'jumlah' and 'price' or 'harga' in text_lower:
        print("This is a valid receipt.")
    else:
        print("This is not a valid receipt.")


def process_image(image):
    # Convert PIL Image to numpy array
    image_np = np.array(image)
    if image_np.dtype != np.uint8:
        image_np = (image_np * 255).astype(np.uint8)  # Normalisasi jika float64

    # Extract text using EasyOCR
    results = reader.readtext(image_np)

    # Combine all detected text into a single string
    extracted_text = ' '.join([result[1] for result in results])

    # Check if the extracted text is a receipt
    is_receipt(extracted_text)

    print("Extracted Text:")
    print(extracted_text)

Fine Tuning

In [ ]:
# data_tuning_local = load_dataset("naver-clova-ix/cord-v2", split="train")
# data_tuning_inter = load_dataset("katanaml-org/invoices-donut-data-v1", split="train")
# data_tuning_inter[5]


In [ ]:
# import os
# import json

# def convert_dataset_local(dataset, output_file):
#     annotations = []

#     # Pastikan folder untuk menyimpan gambar ada
#     os.makedirs("dataset_images", exist_ok=True)

#     for i, data in enumerate(dataset):
#         image = data["image"]  # Objek gambar dari PIL
#         ground_truth = json.loads(data["ground_truth"])  # Parsing JSON string

#         # Simpan gambar ke folder agar bisa digunakan oleh EasyOCR
#         image_path = f"dataset_images/image_{i}.jpg"
#         image.save(image_path)

#         # Ambil teks dan bounding box dari ground truth
#         text = ground_truth["gt_parse"]  # Semua teks dan informasi struk
#         words = []
#         for line in ground_truth["valid_line"]:  # Ambil setiap baris
#             for word in line["words"]:  # Ambil setiap kata dalam baris
#                 words.append({
#                     "text": word["text"],
#                     "quad": word["quad"],
#                     "row_id": word["row_id"],
#                     "category": line["category"],
#                     "group_id": line["group_id"]
#                 })

#         # Simpan dalam format yang sesuai untuk EasyOCR
#         annotations.append({
#             "image": image_path,
#             "text": text,
#             "words": words
#         })

#     # Simpan hasil dalam file JSON
#     with open(output_file, "w", encoding="utf-8") as f:
#         json.dump(annotations, f, indent=4, ensure_ascii=False)


# def convert_dataset_inter(dataset, output_file):
#     """
#     Mengonversi dataset faktur/invoice ke format JSON untuk EasyOCR.
#     """
#     annotations = []

#     for i, data in enumerate(dataset):
#         image = data["image"]  # PIL Image
#         ground_truth = json.loads(data["ground_truth"])["gt_parse"]  # Parse JSON

#         # Simpan gambar
#         image_path = f"dataset_images/invoice_{i}.jpg"
#         os.makedirs("dataset_images", exist_ok=True)
#         image.save(image_path)

#         words = []
#         items = ground_truth.get("items", [])  # Pastikan mengambil 'items' dengan default []
#         if items:
#             for item in items:
#                 words.append({
#                     "text": item.get("item_desc", "UNKNOWN_ITEM"),
#                     "quad": None,
#                     "row_id": None,
#                     "group_id": None
#                 })

#         annotations.append({
#             "image": image_path,
#             "text": ground_truth,  # Simpan seluruh teks yang diparsing
#             "bbox": words
#         })

#         # Pastikan folder Google Drive tujuan benar
#         drive_folder = "G:/My Drive/dataset"
#         output_file = os.path.join(drive_folder, "dataset_annotations.json")

#         # Simpan sebagai JSON
#         with open(output_file, "w", encoding="utf-8") as f:
#             json.dump(annotations, f, indent=4, ensure_ascii=False)

#         print(f"Dataset invoice berhasil disimpan ke {output_file}")


In [ ]:
# # Konversi dataset 1 dan dataset 2
# convert_dataset_local(data_tuning_local, "data_tuning_local.json")
# convert_dataset_inter(data_tuning_inter, "data_tuning_inter.json")

In [ ]:
# python train.py --batch_size 16 --num_epochs 10 --lr 0.001 --train_data data_tuning_local.json data_tuning_inter.json


POST-PROCESSING

Pre-Processing Hasil Text Extraction

Training BERT/NER